## Module 2: Vector Searching

Vector search leverages machine learning to capture the meaning and context of unstructured data, including text and images, transforming it into a numeric representation. Frequently used for semantic search, vector search finds similar data using approximate nearest neighbor algorithms.

Vector search engines - known as vector databases, semantic, or cosine search - find the nearest neighbors to a given (vectorized) query. Where traditional search relies on mentions of keywords, lexical similarity, and the frequency of word occurrences, vector search engines use distances in the embedding space to represent similarity. Finding related data becomes searching for nearest neighbors of the query.

### Qdrant

Qdrant is an open-source vector search engine, a dedicates solution built in Rust for scalable vector search. 

### Install dependencies

In [1]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json
import random
import uuid


In [2]:
qdrant = QdrantClient("http://localhost:6333")

qdrant

### Import the dataset

The dataset has three course types:
- data-engineering-zoomcamp
- machine-learning-zoomcamp
- mlops-zoomcamp

Each course includes a collection of:
- *question:* student's question
- *text:* anwser to student's question
- *section:* course section

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'

response = requests.get(docs_url)
documents_raw = response.json()

documents_raw

### Transform documents into embeddings

As we are building a Q&A RAG system, the `question` and `text` fields will be converted to embeddings to find the most relevant answer to a given question.

The `course` and `section` fields can be stored as metadata to provide more context when the someone wants to ask question related to a specific course or a specific course's section.

#### How to choose the embedding model?

It depends on many factors:
- the task, data modality and specifications
- the trade-off between search precision and resource usage (larger embeddings requeri more storage and memory)
- the cost of inference third-party provider

FastEmbed is an optimized embedding solition designed for Qdrant. It supports:
- dense embeddings for text and images (the most common type in vector search)
- sparse embeddings
- multivector embeddings
- rerankers

FastEmbed's integration with Qdrant allows to directly pass text or images to the Qdrant client for embedding.

#### Find the most suitable embedding model

- Use a small embedding model (e.g. 515 dimensions) and suitable for english text.
- Unimodal model once we are not including images in the search, only text.

In [ ]:
EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent = 2))

embedding_model = "jinaai/jina-embeddings-v2-small-en"

### Create a Qdrant Collection

A *Collection* in Qdrant is a container with a set of data points that belongs to a specific domain/entity we want to search for. It has:
- **Name**: A unique identifier for the collection.
- **Vector size:** The dimensionality of the vector.
- **Distance metric:** The method used to measure similiarity between vectors.


In [ ]:
collection_name = "zoomcamp-rag"

qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

### Create, Embed and Insert a Point into the Collection

A *Point* is the main entity of Qdrant platform. It represents the data point in a high-dimensional space enriched with:
- **Identifier:** A unique identifier for the data point.
- **Vector** (embeddings): It captures the semantic essence of the data.
- **Payload**: It is a JSON object with additioanl metadata. This metadata becomes especially useful when applying filters or sorting during searching.

Before uploading the Points in the Collection, each document is embedded according the selected model (`jinaai/jina-embeddings-v2-small-en`), using the FastEmbed library. The, the generated points will be inserted into the Collection.

In [ ]:
points = []
index = 0


for course in documents_raw:
    for doc in course["documents"]:
        point = models.PointStruct(
            id=index,
            vector=models.Document(text=doc["text"], model=embedding_model),
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"]
            }
        )
        points.append(point)
        index += 1

qdrant.upsert(
    collection_name=collection_name,
    points=points
)

### Perform Semantic Search - Similarity Search

1. Qdrant compares the query's vector to stored vectors, using the distance metric defined previously. 
2. The closest matches are returned, ranked by similarity. 



In [9]:
def search(query, limit=1):
    return qdrant.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=embedding_model
        ),
        limit=limit,
        with_payload=True
    )

In [5]:
course = random.choice(documents_raw)
course_doc = random.choice(course["documents"])
random_question = course_doc["question"]

random_question

'Git - How do I include the files in the Mage repo (including exercise files and homework) in a personal copy of the Data Engineering Zoomcamp repo?'

In [11]:
search_result = search(query=random_question)
search_result

print(f"Question:\n{course_doc['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(search_result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_doc['text']))

Question:
Git - How do I include the files in the Mage repo (including exercise files and homework) in a personal copy of the Data Engineering Zoomcamp repo?

Top Retrieved Answer:
Assuming you downloaded the Mage repo in the week 2 folder of the Data Engineering Zoomcamp, you might want to include your mage copy, demo pipelines and homework within your personal copy of the Data Engineering Zoomcamp repo. This will not work by default, because GitHub sees them as two separate repositories, and one does not track the other. To add the Mage files to your main DE Zoomcamp repo, you will need to:
Move the contents of the .gitignore file in your main .gitignore.
Use the terminal to cd into the Mage folder and:
run “git remote remove origin” to de-couple the Mage repo,
run “rm -rf .git” to delete local git files,
run “git add .” to add the current folder as changes to stage, commit and push.

Original Answer:
Assuming you downloaded the Mage repo in the week 2 folder of the Data Engineering 

In [12]:
search_results = search(query="What if I submit homeworks late?")
answer = search_results.points[0].payload["text"]

answer

'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]'

### Perform Semantic Search - Similarity Search with Filters

Qdrant's custom vector index implementation allows for precise and scalable vector search with filtering conditions.

For example, we can search for an answer related to a specific course. Using the `must` filter ensures that all specified conditions are met for a data point. 

Qdrant also supports other filter types such as `should`, `must_not`, `range`, and more.

To enable efficient filtering, we need to activate the indexing of payload fields.


In [13]:
qdrant.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
def search_by_filter(query, filter_key, filter_value, limit = 1):
    return qdrant.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=embedding_model
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key=filter_key,
                    match=models.MatchValue(value=filter_value)
                )
            ]
        ),
        limit=limit,
        with_payload=True
    )

In [14]:
search_filter_results = search_by_filter(query="What if I submit homeworks late?", filter_key="course", filter_value="mlops-zoomcamp")

answer_filtered = search_filter_results.points[0].payload["text"]

answer_filtered


'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.'

## Implementing RAG with Vector Search

In [ ]:
faq_collection_name = "zoomcamp-faq"

qdrant.create_collection(
    collection_name=faq_collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

In [ ]:
documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

points = []
for index, doc in enumerate(documents):
    text = doc['question'] + " " + doc['text']
    vector = models.Document(text=text, model=embedding_model)
    point = models.PointStruct(
        id=index,
        vector=vector,
        payload=doc
    )

    points.append(point)

In [17]:
qdrant.create_payload_index(
    collection_name=faq_collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [33]:
qdrant.upsert(
    collection_name=faq_collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
faq_question = "I just discovered the course. Can I still join it?"
faq_course = "data-engineering-zoomcamp"

result_points = qdrant.query_points(
    collection_name=faq_collection_name,
    query=models.Document(
        text=faq_question,
        model=embedding_model
    ),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=faq_course)
            )
        ]
    )
)

search_results = []

for point in result_points.points:
    answer = point.payload
    search_results.append(answer)

search_results

## Hybrid Search

Combination of **Semantic search** and **Keyword-based search**.


FastEmbed comes with a BM25 implemetation that we can use to embedding our data points. The data upload is much more fast when compared to dense embedding models, because BM25 algorithm does not require any neural network.



### Insert Data Points using BM25 as embedding model

In [ ]:
sparse_collection_name = "zoomcamp-sparse"

qdrant.create_collection(
    collection_name=sparse_collection_name,
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)


In [ ]:
for course in documents_raw:
    for doc in course["documents"]:
        qdrant.upsert(
            collection_name=sparse_collection_name,
            points=[
                models.PointStruct(
                    id=uuid.uuid4().hex,
                    vector={
                        "bm25": models.Document(
                            text=doc["text"],
                            model="Qdrant/bm25"
                        )},
                    payload={
                        "text": doc["text"],
                        "section": doc["section"],
                        "course": course["course"]
                    }
                )
            ]
        )

### Search with BM25

Sparse vectors can return no results if none of the keywrods from the query were used in the documents. No matter if they are similar. Semantic does not matter, only terminology.

In [6]:
def search(query, limit=1):
    results = qdrant.query_points(
        collection_name=sparse_collection_name,
        query=models.Document(
            text=query,
            model="Qdrant/bm25"
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )

    return results.points

In [7]:
search("Qdrant")

[]

In [8]:
results = search("pandas")

results[0].payload["text"]

"You can use round() function or f-strings\nround(number, 4)  - this will round number up to 4 decimal places\nprint(f'Average mark for the Homework is {avg:.3f}') - using F string\nAlso there is pandas.Series. round idf you need to round values in the whole Series\nPlease check the documentation\nhttps://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round\nAdded by Olga Rudakova"

### Multi-Stage Queries

There are use-cases when the best search is obtained by combining multiple queries, or by performning in more than one stage.

The main component for making the combinations of queries possible is the **prefetch** parameter, which enables making sub-requests.

Whenever a query has at least one prefetch, Qdrant will:

1. Perform the prefetch query 
2. Apply the main query over the results of its prefetch(es)


In [ ]:
sparse_dense_collection_name = "zoomcamp-sparse-and-dense"

qdrant.create_collection(
    collection_name=sparse_dense_collection_name,
    vectors_config={
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

In [27]:
for course in documents_raw:
    for doc in course["documents"]:
        qdrant.upsert(
            collection_name=sparse_dense_collection_name,
            points=[
                models.PointStruct(
                    id=uuid.uuid4().hex,
                    vector={
                        "jina-small": models.Document(
                            text=doc["text"],
                            model="jinaai/jina-embeddings-v2-small-en"
                        ),
                        "bm25": models.Document(
                            text=doc["text"],
                            model="Qdrant/bm25"
                        )
                    },
                    payload={
                        "text": doc["text"],
                        "section": doc["section"],
                        "course": course["course"]
                    }
                )
            ]
        )

In [9]:
def multi_stage_search(query, limit=1):
    results = qdrant.query_points(
        collection_name=sparse_dense_collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit)
            )
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25"
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )

    return results.points

In [16]:
random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [13]:
results = multi_stage_search(course_piece["question"])

results[0].payload["text"]

"Problem description. How can we connect s3 bucket to MLFLOW?\nSolution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need access keys because AWS won't care.\nRead more here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html\nAdded by Akshit Miglani"

### Hybrid Search and Fusion

Hybrid Search is a technique for combining results coming from different search methods - dense and sparse. Dense and sparse search scores cannot be compared directly, so we need another method that will order the final results somehow.

There are two important terms for it:
- **Fusion**
- **Reranking**

#### Fusion 
Fusion is a set of methods which work on the scores/ranking as returned by the individual methods. There are many ways of how to achieve that, but **Reciprocal Rank Fusion** is the most popular one. It is based on the rankings of the documents in each methods (dense and sparse ranking), and these rankings are used to calculate the final scores. The final ranking is calculated based on the RRF score.



In [21]:
# Reciprocal Rank Fusion search

def rrf_search(query, limit=1):
    results = qdrant.query_points(
        collection_name=sparse_dense_collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model= "jinaai/jina-embeddings-v2-small-en"
                ),
                using="jina-small",
                limit=(5 * limit)
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25"
                    ),
                using="bm25",
                limit=(5 * limit)
                )
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True
    )

    return results.points

In [23]:
results = rrf_search(course_piece["question"])
print(json.dumps(course_piece, indent=2))
print(results[0].payload["text"])

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}
Problem description. How can we connect s3 bucket to MLFLOW?
Solution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need a